In [1]:
import pandas as pd
import requests


### xml을 DataFrame으로 변환하기 ###
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

# 모듈 import
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
from xml.etree.ElementTree import parse
import xmltodict


In [2]:
def schedule_api():
    input_year = int(input('몇 년도의 운항 스케줄을 뽑으시겠습니까?: '))
    # 날짜(범위로) 생성해주는 코드
    date_data = []
    from datetime import date, timedelta

    def daterange(start_date, end_date):
        for n in range(int((end_date - start_date).days)):
            yield start_date + timedelta(n)

            
            
    if input_year == 2022:
        start_date = date(input_year, 1, 1)
        end_date = date(input_year, 7, 31)
    else:
            start_date = date(input_year, 1, 1)
        end_date = date(input_year+1, 1, 31)
        

    for single_date in daterange(start_date, end_date):
        date_data.append(single_date.strftime("%Y%m%d"))
    url = 'http://openapi.airport.co.kr/service/rest/FlightScheduleList/getDflightScheduleList'
    api_key = "7GRMBFtqXjvKY%2FctwLPOnFYQKe%2BLd50fIjkSxBfWMl8nrYB3bkyfjjj9B3Be3GwvolJVtVf9t%2BWMet9bewRegA%3D%3D"
    api_key_decode = requests.utils.unquote(api_key)

    airline = ''
    air_num = ''

    
    
    
 


    # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
    
    row_list = [] # 행값
    name_list = ['airlineEnglish',
     'airlineKorean',
     'arrivalcity',
     'arrivalcityCode',
     'domesticArrivalTime',
     'domesticEddate',
     'domesticFri',
     'domesticMon',
     'domesticNum',
     'domesticSat',
     'domesticStartTime',
     'domesticStdate',
     'domesticSun',
     'domesticThu',
     'domesticTue',
     'domesticWed',
     'flightPurpose',
     'startcity',
     'startcityCode']
    value_list = [] #데이터값

    print('##### 1. 출발 기준 데이터 수집을 시작합니다 #######')
    print()
    dept_city = 'GMP'
    arrv_city = 'PUS'
    for d in range(2):

        for date in date_data: 
            params ={'serviceKey' : api_key_decode, 'schDate' : date, 'schDeptCityCode' : dept_city, 'schArrvCityCode' : arrv_city, 'schAirLine' : airline, 'schFlightNum' : air_num, 'numOfRows' : 35299}
            req = requests.get(url, params = params)
            content = req.text 

            #bs4 사용하여 item 태그 분리
            xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
            rows = xml_obj.findAll('item')

            # xml 안의 데이터 수집
            for i in range(0, len(rows)):
                columns = rows[i].find_all()
                #첫째 행 데이터 수집
                for j in range(0,len(columns)):
                    # 컬럼의 각 데이터 값 저장
                    value_list.append(columns[j].text)
                # 각 행의 value값 전체 저장
                row_list.append(value_list)
                # 데이터 리스트 값 초기화
                value_list=[]
            print('###### {} 일자 데이터 수집 완료 ######'.format(date))
        dept_city = 'PUS'
        arrv_city = 'GMP'
        print()
        if d == 0:
            print('##### 2. 도착 기준 데이터 수집을 시작합니다 #######')
            print()
        else:
            print('{}년 기준 {} 일자까지의 항공 스케줄 데이터 수집을 완료했습니다.'.format(input_year, date))


    #xml값 DataFrame으로 만들기
    airline_df = pd.DataFrame(row_list, columns=name_list)

    # airline_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    airline_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
    # 운항일자순으로 정렬
    airline_df.sort_values(by='domesticStdate', inplace=True)
    airline_df.reset_index(drop=True, inplace=True)

    airline_df = airline_df[['domesticNum', 'airlineKorean', 'startcityCode', 'arrivalcityCode', 'domesticStartTime', 'domesticArrivalTime', 'domesticStdate']]

    airline_df.rename({'domesticNum':'운항편명', 'airlineKorean':'항공사명', 'startcityCode': '출발공항', 'arrivalcityCode': '도착공항','domesticStartTime':'출발예정시간',
              'domesticArrivalTime':'도착예정시간', 'domesticStdate':'운항일자'}, axis=1, inplace=True)
    
    return airline_df, input_year

In [3]:
schedule_data, input_year = schedule_api()

몇 년도의 운항 스케줄을 뽑으시겠습니까?: 2022
##### 1. 출발 기준 데이터 수집을 시작합니다 #######

###### 20220101 일자 데이터 수집 완료 ######
###### 20220102 일자 데이터 수집 완료 ######
###### 20220103 일자 데이터 수집 완료 ######
###### 20220104 일자 데이터 수집 완료 ######
###### 20220105 일자 데이터 수집 완료 ######
###### 20220106 일자 데이터 수집 완료 ######
###### 20220107 일자 데이터 수집 완료 ######
###### 20220108 일자 데이터 수집 완료 ######
###### 20220109 일자 데이터 수집 완료 ######
###### 20220110 일자 데이터 수집 완료 ######
###### 20220111 일자 데이터 수집 완료 ######
###### 20220112 일자 데이터 수집 완료 ######
###### 20220113 일자 데이터 수집 완료 ######
###### 20220114 일자 데이터 수집 완료 ######
###### 20220115 일자 데이터 수집 완료 ######
###### 20220116 일자 데이터 수집 완료 ######
###### 20220117 일자 데이터 수집 완료 ######
###### 20220118 일자 데이터 수집 완료 ######
###### 20220119 일자 데이터 수집 완료 ######
###### 20220120 일자 데이터 수집 완료 ######
###### 20220121 일자 데이터 수집 완료 ######
###### 20220122 일자 데이터 수집 완료 ######
###### 20220123 일자 데이터 수집 완료 ######
###### 20220124 일자 데이터 수집 완료 ######
###### 20220125 일자 데이터 수집 완료 ######
###### 20220126 일자 데이터 수집 완료 ####

###### 20220815 일자 데이터 수집 완료 ######
###### 20220816 일자 데이터 수집 완료 ######
###### 20220817 일자 데이터 수집 완료 ######
###### 20220818 일자 데이터 수집 완료 ######
###### 20220819 일자 데이터 수집 완료 ######
###### 20220820 일자 데이터 수집 완료 ######
###### 20220821 일자 데이터 수집 완료 ######
###### 20220822 일자 데이터 수집 완료 ######
###### 20220823 일자 데이터 수집 완료 ######
###### 20220824 일자 데이터 수집 완료 ######
###### 20220825 일자 데이터 수집 완료 ######
###### 20220826 일자 데이터 수집 완료 ######
###### 20220827 일자 데이터 수집 완료 ######
###### 20220828 일자 데이터 수집 완료 ######
###### 20220829 일자 데이터 수집 완료 ######
###### 20220830 일자 데이터 수집 완료 ######
###### 20220831 일자 데이터 수집 완료 ######
###### 20220901 일자 데이터 수집 완료 ######
###### 20220902 일자 데이터 수집 완료 ######
###### 20220903 일자 데이터 수집 완료 ######
###### 20220904 일자 데이터 수집 완료 ######
###### 20220905 일자 데이터 수집 완료 ######
###### 20220906 일자 데이터 수집 완료 ######
###### 20220907 일자 데이터 수집 완료 ######
###### 20220908 일자 데이터 수집 완료 ######
###### 20220909 일자 데이터 수집 완료 ######
###### 20220910 일자 데이터 수집 완료 ######
###### 20220911 일자 데이터 수집 완료

###### 20220331 일자 데이터 수집 완료 ######
###### 20220401 일자 데이터 수집 완료 ######
###### 20220402 일자 데이터 수집 완료 ######
###### 20220403 일자 데이터 수집 완료 ######
###### 20220404 일자 데이터 수집 완료 ######
###### 20220405 일자 데이터 수집 완료 ######
###### 20220406 일자 데이터 수집 완료 ######
###### 20220407 일자 데이터 수집 완료 ######
###### 20220408 일자 데이터 수집 완료 ######
###### 20220409 일자 데이터 수집 완료 ######
###### 20220410 일자 데이터 수집 완료 ######
###### 20220411 일자 데이터 수집 완료 ######
###### 20220412 일자 데이터 수집 완료 ######
###### 20220413 일자 데이터 수집 완료 ######
###### 20220414 일자 데이터 수집 완료 ######
###### 20220415 일자 데이터 수집 완료 ######
###### 20220416 일자 데이터 수집 완료 ######
###### 20220417 일자 데이터 수집 완료 ######
###### 20220418 일자 데이터 수집 완료 ######
###### 20220419 일자 데이터 수집 완료 ######
###### 20220420 일자 데이터 수집 완료 ######
###### 20220421 일자 데이터 수집 완료 ######
###### 20220422 일자 데이터 수집 완료 ######
###### 20220423 일자 데이터 수집 완료 ######
###### 20220424 일자 데이터 수집 완료 ######
###### 20220425 일자 데이터 수집 완료 ######
###### 20220426 일자 데이터 수집 완료 ######
###### 20220427 일자 데이터 수집 완료

###### 20221114 일자 데이터 수집 완료 ######
###### 20221115 일자 데이터 수집 완료 ######
###### 20221116 일자 데이터 수집 완료 ######
###### 20221117 일자 데이터 수집 완료 ######
###### 20221118 일자 데이터 수집 완료 ######
###### 20221119 일자 데이터 수집 완료 ######
###### 20221120 일자 데이터 수집 완료 ######
###### 20221121 일자 데이터 수집 완료 ######
###### 20221122 일자 데이터 수집 완료 ######
###### 20221123 일자 데이터 수집 완료 ######
###### 20221124 일자 데이터 수집 완료 ######
###### 20221125 일자 데이터 수집 완료 ######
###### 20221126 일자 데이터 수집 완료 ######
###### 20221127 일자 데이터 수집 완료 ######
###### 20221128 일자 데이터 수집 완료 ######
###### 20221129 일자 데이터 수집 완료 ######
###### 20221130 일자 데이터 수집 완료 ######
###### 20221201 일자 데이터 수집 완료 ######
###### 20221202 일자 데이터 수집 완료 ######
###### 20221203 일자 데이터 수집 완료 ######
###### 20221204 일자 데이터 수집 완료 ######
###### 20221205 일자 데이터 수집 완료 ######
###### 20221206 일자 데이터 수집 완료 ######
###### 20221207 일자 데이터 수집 완료 ######
###### 20221208 일자 데이터 수집 완료 ######
###### 20221209 일자 데이터 수집 완료 ######
###### 20221210 일자 데이터 수집 완료 ######
###### 20221211 일자 데이터 수집 완료

In [6]:
schedule_data

,운항편명,항공사명,출발공항,도착공항,출발예정시간,도착예정시간,운항일자
0,OZ8902,아시아나항공,CJU,GMP,0700,0810,2021-10-31T00:00:00+09:00
1,LJ323,진에어,GMP,CJU,1745,1900,2021-10-31T00:00:00+09:00
2,RS907,에어서울,GMP,CJU,1750,1910,2021-10-31T00:00:00+09:00
3,RS907,에어서울,GMP,CJU,1750,1905,2021-10-31T00:00:00+09:00
4,LJ312,진에어,CJU,GMP,1145,1255,2021-10-31T00:00:00+09:00
...,...,...,...,...,...,...,...
2922,TW9709,티웨이항공,GMP,CJU,1035,1145,2022-12-04T00:00:00+09:00
2923,TW9709,티웨이항공,GMP,CJU,1030,1140,2022-12-05T00:00:00+09:00
2924,BX8011,에어부산,GMP,CJU,0650,0805,2022-12-10T00:00:00+09:00
2925,RS901,에어서울,GMP,CJU,0705,0810,2022-12-30T00:00:00+09:00


In [7]:
# 2018년일 경우 2017년 말 운항일짜가 나올 수도 있음(다른 연도도 해당) 나중에 엑셀에서 전처리 필요 
# -> 만약 2019년 데이터에 운행일자가 2018년도가 나오면 해당 데이터를 2018년 데이터로 이동시키자 
    
#  엑셀 저장 코드 
schedule_data.to_excel('1. {}년 항공 스케줄 데이터(API).xlsx'.format(input_year), index=False)